In [1]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
db_pomoci = pd.read_csv("db_pomoci.csv")

In [2]:
black_list = ['http://www.dc-brno.cz']
df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev"})
df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
# df['web'] = df['web'].str.replace('https://www', 'www')
# df['web'] = df['web'].str.replace('www', 'https://www')
df_agg = df.groupby("web")["Telefon"].agg(list)
df_agg["email"] = df.groupby("web")["E-mail"].agg(list)
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)
#df.sort_values("web")


In [19]:
df[df["web"].str.contains('brno')]

,nazev,Kategorie,Adresa,E-mail,Telefon,web
63,CDZ Brno I (Práh jižní Morava),CDZ - Centrum duševního zdraví,"Štěpánská 2, Brno-Trnitá (část) 602 00, Jihomo...",terennitym@prah-brno.cz,733 643 379,https://www.cdz-brno.cz/
64,CDZ Brno II (Práh jižní Morava),CDZ - Centrum duševního zdraví,"Jugoslávská 17, Zábrdovice (Brno-sever) (část)...",cdz@pnbrno.cz,737 390 737,https://www.pnbrno.cz/centrum-dusevniho-zdravi/
321,Odborné sociální poradenství - poradna PřES PRÁH,Odborné sociální poradenství,"Tuřanská 12, Brno-Tuřany (část) 620 00, Jihomo...",jana.wudyova@prah-brno.cz,734 850 546,http://www.prah-brno.cz/novinky/2-uncategorise...
492,"Pedagogicko-psychologická poradna Brno, příspě...",Pedagogicko-psychologické poradny,"Hybešova 253/15, Staré Brno, 602 00 Brno",hybesova@pppbrno.cz,541 422 822,https://www.pppbrno.cz/cs/
546,"Černý Vojtěch PhDr., Ph.D.",Psychoterapeuti,"Františkánská 6, 602 00 Brno",info@cervoy.cz,602 737 026,http://www.psychologiebrno.cz/
1428,Nízkoprahový klub Kumbál,Nízkoprahové zařízení,"Vídeňská 14/78, Brno-střed, Štýřice, 639 00 Brno",gajdziok.jan@brno.cz,543 210 670,http://www.brno.cz


In [3]:
urls = set(df["web"])


In [5]:

# Regular expressions for matching emails and phone numbers
email_regex = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
phone_regex = re.compile(r"\+?(\d[\d\-\(\) ]{9,}\d)")

# List to collect data
data = []

# Function to scrape a single URL
def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        return emails, phones
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return set(), set()

# Loop through the URLs and scrape each one
for url in tqdm(urls):
    print(f"Scraping {url}")
    
    emails, phones = scrape_website(url)
    # Convert sets to comma-separated strings for storage
    #emails_str = ', '.join(emails)
    phones_str = ', '.join(phones)
    
    # Collect data
    #data.append({'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str})
    data.append({'Website': url, 'Phone Numbers': phones_str})
    print("Done")
    print("")
# Create DataFrame from collected data
df_res = pd.DataFrame(data)

print(df)

  0%|          | 0/870 [00:00<?, ?it/s]

Scraping https://www.zachranny-kruh.cz/


  0%|          | 2/870 [00:01<08:42,  1.66it/s]

Done

Scraping http://psych.fnplzen.cz/cs/node/839
Done

Scraping http://www.porceta.cz
Error fetching http://www.porceta.cz: 403 Client Error: Forbidden for url: https://www.porceta.cz/
Done

Scraping https://www.prevent99.cz/spot/


  1%|          | 6/870 [00:01<02:53,  4.98it/s]

...

Done

Scraping http://www.dejdar-martin.cz/
Done

Scraping https://www.diakoniezapad.cz/sluzby-diakonie-zapad/pro-lidi-v-nouzi/poradna-pro-obcany-v-nesnazich/


  9%|▉         | 79/870 [01:04<15:15,  1.16s/it]

Error fetching https://www.diakoniezapad.cz/sluzby-diakonie-zapad/pro-lidi-v-nouzi/poradna-pro-obcany-v-nesnazich/: 404 Client Error: Not Found for url: https://www.zapad.diakonie.cz/sluzby-diakonie-zapad/pro-lidi-v-nouzi/poradna-pro-obcany-v-nesnazich/
Done

Scraping http://ppp3a9.cz/
Error fetching http://ppp3a9.cz/: HTTPConnectionPool(host='ppp3a9.cz', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff4e855b460>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Done

Scraping https://www.freeklub.cz


  9%|▉         | 80/870 [01:24<13:56,  1.06s/it]


KeyboardInterrupt: 

In [6]:
df_res = pd.DataFrame(data)

In [7]:
df_res

,Website,Phone Numbers
0,https://www.zachranny-kruh.cz/,1 - 197-197
1,http://psych.fnplzen.cz/cs/node/839,"377 103 959, 377 441 850, 377 103 111, 377 401..."
2,http://www.porceta.cz,
3,https://www.prevent99.cz/spot/,
4,http://www.detemzlin.cz/,577 922 056
...,...,...
75,https://www.firmy.cz/detail/2422937-mudr-veron...,420 739 122 265
76,https://www.firmy.cz/detail/12999364-phdr-eva-...,"725 048 038, 420 774 230 134"
77,http://www.dejdar-martin.cz/,607 535 595
78,https://www.diakoniezapad.cz/sluzby-diakonie-z...,


In [8]:
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)


In [9]:
df_res = df_res.rename(columns= {"Website":"web"})

In [10]:
df_agg.drop("index", axis = 1,inplace=True)
df_agg.drop("level_0",axis = 1, inplace=True)
df_agg.info()

KeyError: "['level_0'] not found in axis"

In [11]:
preview = df_res.merge(df_agg , on="web")

In [15]:
preview.head(30)

,web,Phone Numbers,Telefon
0,https://www.zachranny-kruh.cz/,1 - 197-197,[353 230 000]
1,http://psych.fnplzen.cz/cs/node/839,"377 103 959, 377 441 850, 377 103 111, 377 401...",[377 103 111]
2,http://www.porceta.cz,,[777 882 002]
3,https://www.prevent99.cz/spot/,,[606 089 789]
4,http://www.detemzlin.cz/,577 922 056,[577 645 231]
5,https://www.homolka.cz/,,[257 271 111]
6,http://www.educozatec.cz/,,[603 253 634]
7,https://www.charitasluknov.cz,"602 236 279, 723 352 633","[734 799 081, 733 160 911]"
8,http://www.psychoterapeuti.cz/adresar-psychote...,"774 979 714, 257 322 366, 770 623 916, 607 669...","[233 351 741, 776 736 832]"
9,http://www.psychoterapeuti.cz/adresar-psychote...,"774 979 714, 257 322 366, 770 623 916, 607 669...",[777 808 424]
